In [ ]:
import nltk
from nltk.corpus import words
import random
from wordle import WordleClone

In [ ]:
# get letter frequency by position
def init_word_list():
    nltk.download('words', quiet=True)
    return [word.upper() for word in words.words() if len(word) == 5 and word.isalpha()]

def get_letter_freq(word_list):
    letter_freq = {}
    for word in words:
        for letter in word:
            if letter in letter_freq:
                letter_freq[letter] += 1
            else:
                letter_freq[letter] = 1
    return sort_letter_freq_dict(letter_freq)

def get_letter_freq_by_pos(word_list):
    pos_freq = [{}, {}, {}, {}, {}]
    for w in word_list:
        for i in range(5):
            if w[i] in pos_freq[i]:
                pos_freq[i][w[i]] += 1
            else:
                pos_freq[i][w[i]] = 1
    return pos_freq

def sort_letter_freq_dict(letter_freq_dict):
    rslt = [(k, v) for k, v in letter_freq_dict.items()]
    return sorted(rslt, key=lambda x: x[1], reverse=True)

# filter by known letter location
def filter_by_letter_loc(letter, location, word_list):
    return [_ for _ in word_list if _[location] == letter]

# filter by known letter, deny-list location
def filter_by_letter_neg_loc(letter, neg_loc_list, word_list):
    for loc in neg_loc_list:
        word_list = [_ for _ in word_list if letter in _ and _[loc] != letter]
    return word_list

def filter_by_unused_letter(letter, word_list):
    return [_ for _ in word_list if letter not in _]

In [ ]:
def test_filter_by_letter_loc():
    print('testing filter_by_letter_loc')
    print(filter_by_letter_loc('A', 0, ['ABBBB', 'BAAAA']) == ['ABBBB'])
    print(filter_by_letter_loc('A', 1, ['ABBBB', 'BAAAA']) == ['BAAAA'])
    print(filter_by_letter_loc('C', 1, ['ABBBB', 'BAAAA']) == [])

def test_filter_by_letter_neg_loc():
    print('testing filter_by_letter_neg_loc')
    print(filter_by_letter_neg_loc('A', [0], ['ABCDE', 'BACDE']) == ['BACDE'])
    print(filter_by_letter_neg_loc('A', [0, 1], ['ABCDE', 'BACDE']) == [])
    print(filter_by_letter_neg_loc('F', [0, 1], ['ABCDE', 'BACDE']) == [])

test_filter_by_letter_loc()
test_filter_by_letter_neg_loc()

In [ ]:
get_letter_freq(words)

In [ ]:
get_letter_freq_by_pos(words)

In [ ]:
get_letter_freq_by_pos(words)

# Bootstrap
What letters are most commonly used overall?
What letters give you the most information as an opening gambit?

In [ ]:
def generate_guess_word(word_list):
    start_word_list = []
    for i in range(5):
        letter_freq = get_letter_freq_by_pos(word_list)
        letter_freq_list = sort_letter_freq_dict(letter_freq[i])
        letter_freq_list = [_ for _ in letter_freq_list if _[0] not in start_word_list]
        letter_choice = random.choice([_[0] for _ in letter_freq_list][:3]) 
        start_word_list.append(letter_choice)
        word_list = filter_by_letter_loc(letter_choice, i, word_list)
    return ''.join(start_word_list)

def get_guess_word(word_list):
    for attempt in range(5):
        try:
            return generate_guess_word(word_list)
        except Exception as e:
            continue

# Play the game

In [ ]:
def apply_feedback_to_word_list(guess, feedback, word_list):
    for i in range(5):
        if feedback[i] == 'G':
            word_list = filter_by_letter_loc(guess[i], i, word_list)
        elif feedback[i] == 'Y':
            word_list = filter_by_letter_neg_loc(guess[i], [i], word_list)
        else:
            word_list = filter_by_unused_letter(guess[i], word_list)
    return word_list

In [ ]:
def play_wordle():
    viable_words = init_word_list()
    wc = WordleClone()
    while True:
        guess = get_guess_word(viable_words)
        print(guess)
        feedback = wc.guess_word(guess)
        print(feedback)
        if isinstance(feedback, str) and feedback.startswith('Congratulations!'):
            print('Congratulations!')
            return True
        elif isinstance(feedback, str) and feedback.startswith('Game Over!'):
            print('Game Over!')
            return False
        viable_words = apply_feedback_to_word_list(guess, feedback, viable_words)

In [ ]:
play_wordle()